In [3]:
import asyncio
from langchain_ollama import OllamaLLM
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# =============== CONFIG ===============
OLLAMA_MODEL = "qwen3:latest"  # or "gemma2", "mistral", etc.
search = GoogleSearchAPIWrapper()

# =============== TOOLS ===============
google_search_tool = Tool(
    name="Google Search",
    func=search.run,
    description="Search Google for the latest information on a topic."
)
# =============== SUB-AGENT PROMPTS ===============

research_prompts = {
    "renewable_energy": PromptTemplate(
        input_variables=["topic"],
        template="""You are an AI Research Assistant specializing in energy.
Research the latest advancements in 'renewable energy sources'.
Use the Google Search tool provided.
Summarize your key findings concisely (1-2 sentences).
Output only the summary."""
    ),
    "electric_vehicles": PromptTemplate(
        input_variables=["topic"],
        template="""You are an AI Research Assistant specializing in transportation.
Research the latest developments in 'electric vehicle technology'.
Use the Google Search tool provided.
Summarize your key findings concisely (1-2 sentences).
Output only the summary."""
    ),
    "carbon_capture": PromptTemplate(
        input_variables=["topic"],
        template="""You are an AI Research Assistant specializing in climate solutions.
Research the current state of 'carbon capture methods'.
Use the Google Search tool provided.
Summarize your key findings concisely (1-2 sentences).
Output only the summary."""
    )
}

# =============== CREATE RESEARCHER CHAINS ===============

llm = OllamaLLM(model=OLLAMA_MODEL)

renewable_chain = LLMChain(prompt=research_prompts["renewable_energy"], llm=llm)
ev_chain = LLMChain(prompt=research_prompts["electric_vehicles"], llm=llm)
carbon_chain = LLMChain(prompt=research_prompts["carbon_capture"], llm=llm)

# =============== RUN PARALLEL RESEARCH ===============

async def run_parallel_research():
    results = await asyncio.gather(
        renewable_chain.arun(topic="renewable energy"),
        ev_chain.arun(topic="electric vehicles"),
        carbon_chain.arun(topic="carbon capture")
    )

    return {
        "renewable_energy_result": results[0],
        "ev_technology_result": results[1],
        "carbon_capture_result": results[2],
    }

# =============== SYNTHESIS AGENT ===============

synthesis_prompt = PromptTemplate(
    input_variables=[
        "renewable_energy_result",
        "ev_technology_result",
        "carbon_capture_result"
    ],
    template="""## Summary of Recent Sustainable Technology Advancements

### Renewable Energy Findings
(Based on RenewableEnergyResearcher's findings)
{renewable_energy_result}

### Electric Vehicle Findings
(Based on EVResearcher's findings)
{ev_technology_result}

### Carbon Capture Findings
(Based on CarbonCaptureResearcher's findings)
{carbon_capture_result}

### Overall Conclusion
Provide a brief (1-2 sentence) concluding statement that connects only the findings above."""
)

synthesis_chain = LLMChain(prompt=synthesis_prompt, llm=llm)

# =============== ORCHESTRATOR ===============

async def main():
    # Step 1: Run all research agents in parallel
    research_results = await run_parallel_research()

    # Step 2: Run the synthesis step
    final_report = await synthesis_chain.arun(**research_results)

    print("\n===== FINAL SYNTHESIZED REPORT =====\n")
    print(final_report)

# Run the workflow
if __name__ == "__main__":
    asyncio.run(main())


ValidationError: 1 validation error for GoogleSearchAPIWrapper
  Value error, Did not find google_api_key, please add an environment variable `GOOGLE_API_KEY` which contains it, or pass `google_api_key` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

In [2]:
pip install -U langchain-google-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 13.2 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 16.7 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19/19 [langchain-google-community]i-python-client]
Note: you may need to restart the kernel to use updated packages.
